<a href="https://colab.research.google.com/github/huyhoangttql/Reviews_film_classification_using_ANN/blob/main/Imdb_classifer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)

Version:  2.4.1
Eager mode:  True
Hub version:  0.11.0


In [3]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBUXIVA/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBUXIVA/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteBUXIVA/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
train_examples_batch,train_labels_batch = next(iter(train_data.batch(10)))

In [7]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [8]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [11]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 6s 87ms/step - loss: 0.6945 - accuracy: 0.5048 - val_loss: 0.6392 - val_accuracy: 0.5572
Epoch 2/10
30/30 [==============================] - 3s 82ms/step - loss: 0.6089 - accuracy: 0.5842 - val_loss: 0.5542 - val_accuracy: 0.6542
Epoch 3/10
30/30 [==============================] - 3s 84ms/step - loss: 0.4996 - accuracy: 0.7337 - val_loss: 0.4481 - val_accuracy: 0.7872
Epoch 4/10
30/30 [==============================] - 3s 84ms/step - loss: 0.3651 - accuracy: 0.8477 - val_loss: 0.3649 - val_accuracy: 0.8389
Epoch 5/10
30/30 [==============================] - 3s 86ms/step - loss: 0.2552 - accuracy: 0.9015 - val_loss: 0.3289 - val_accuracy: 0.8476
Epoch 6/10
30/30 [==============================] - 3s 83ms/step - loss: 0.1869 - accuracy: 0.9348 - val_loss: 0.3122 - val_accuracy: 0.8623
Epoch 7/10
30/30 [==============================] - 3s 82ms/step - loss: 0.1289 - accuracy: 0.9617 - val_loss: 0.3091 - val_accuracy: 0.8680
Epoch 8/10
30

In [12]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3620 - accuracy: 0.8510
loss: 0.362
accuracy: 0.851
